In [1]:
import pymongo
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element

In [2]:
client = MongoClient()
db = client.companies
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

### Based on the information provided, we are going to select a possible location for the office based on the available information.

The applied filter assums a **number of emplooyees of 90**. 

The reason? Currently, a huge number of companies have implanted a ***work from home policy*** but, additionally the company ***expect to increase their employees number in the future***, so being continously changing the location would be counterproductive.

Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [3]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city":{"$ne": "",}}
project1 = {"_id":0,"offices.city": 1,"number_of_employees": 1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)#.skip(1).limit(1)
results1

In [4]:
pd.DataFrame.from_dict(results1)

,number_of_employees,offices
0,90,{'city': 'Amsterdam'}
1,90,{'city': 'San Francisco'}
2,90,{'city': 'Singapore'}
3,90,{'city': 'Denver'}
4,90,{'city': 'San Francisco'}
5,90,{'city': 'Palo Alto'}
6,90,{'city': 'Chicago'}
7,90,{'city': 'San Francisco'}
8,90,{'city': 'Irving'}
9,90,{'city': 'South San Francisco'}


Additionaly, we have selected **New York as the City** to locate the office as it is the USA city with more **Starbucks**.

In [5]:
filt1 = {"number_of_employees": {"$eq":90},"offices.city": {'$eq': "New York"}}
project1 = {"_id":0,"name":1,"offices.city": 1,"number_of_employees": 1,"offices.latitude":1,"offices.longitude":1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)#.skip(1).limit(1)
results1

In [6]:
office_location = list(results1)
office_location

[{'name': 'Aleri',
  'number_of_employees': 90,
  'offices': {'city': 'New York',
   'latitude': 40.7592189,
   'longitude': -73.9783534}}]

In [7]:
pd.DataFrame.from_dict(office_location)

,name,number_of_employees,offices
0,Aleri,90,"{'city': 'New York', 'latitude': 40.7592189, '..."


## OFFICE LOCATION

In [8]:
office_lat = 40.7592189
office_long = -73.9783534

In [9]:
office_data = [("office","40.7592189","-73.9783534","POINT (-73.9783534 40.7592189)","office location")]
df_office = pd.DataFrame(office_data, columns = ["name","latitud","longitud","geometry","criteria"]) 
df_office

,name,latitud,longitud,geometry,criteria
0,office,40.7592189,-73.9783534,POINT (-73.9783534 40.7592189),office location


In [10]:
map_1 = folium.Map(location = [office_lat, office_long], zoom_start = 16)
map_1

In [11]:
office_loc = folium.Marker(location = [office_lat, office_long], tooltip = "Office location proposal")
# tooltip nos sirve para crear texto en los marcadores
office_loc.add_to(map_1)
map_1

icono = Icon(color = "blue",
             prefix = "fa",
             icon = "building-o",
             icon_color = "black",
             tooltip = "Office location proposal")

In [12]:
office_location = [office_lat, office_long]
marker_office = Marker(location = office_location, icon = icono)
marker_office.add_to(map_1)
map_1

In [13]:
type(office_location)

list

In [14]:
load_dotenv()

True

In [15]:
city = "New York"
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":[float(data["longt"]),float(data["latt"])]}
    except:
        return data

In [16]:
donde = "New York"
data = requests.get(f"https://geocode.xyz/{donde}?json=1").json()

In [17]:
print(data)

{'standard': {'addresst': {}, 'city': 'New York', 'prov': 'US', 'countryname': 'United States of America', 'postal': {}, 'confidence': '0.90'}, 'longt': '-73.95861', 'alt': {'loc': {'longt': '-73.9536869565217', 'prov': 'NY', 'city': 'NEW YORK', 'postal': '10075', 'score': '25', 'latt': '40.7724386956522'}}, 'elevation': {}, 'latt': '40.68908'}


In [18]:
nyc = geocode(city)

In [19]:
nyc

{'type': 'Point', 'coordinates': [-73.95861, 40.68908]}

In [20]:
nyc = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}

In [21]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

### Búsqueda de Starbucks

In [22]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
starbucks = "556f676fbd6a75a99038d8ec"

In [23]:
parametros = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"Starbucks",
              "radius":500
}

In [24]:
resp = requests.get(url= url_query, params=parametros)
data = json.loads(resp.text)

In [25]:
data.keys()

dict_keys(['meta', 'response'])

In [26]:
decoding_data = data.get("response")

In [27]:
decoded = decoding_data.get("groups")[0]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
starbucks = decoded.get("items")

In [ ]:
map_starbucks = ["venue","name"]
m_latitud = ["venue","location","lat"]
m_longitud = ["venue","location","lng"]

In [ ]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [ ]:
print(getFromDict(starbucks[0],map_starbucks))

In [ ]:
starbucks_nyc = []
for dic in starbucks:
    paralista = {}
    paralista["name"] = getFromDict(dic,map_starbucks)
    paralista["latitud"] = getFromDict(dic,m_latitud)
    paralista["longitud"] = getFromDict(dic,m_longitud)
    starbucks_nyc.append(paralista)

In [ ]:
starbucks_nyc[0]

In [ ]:
df_starbucks = pd.DataFrame(starbucks_nyc)
df_starbucks.head()

In [ ]:
gdf_starbucks = gpd.GeoDataFrame(df_starbucks, geometry = gpd.points_from_xy(df_starbucks.longitud,df_starbucks.latitud))
gdf_starbucks.head()

In [ ]:
gdf_starbucks = gdf_starbucks.assign(name = [ "Starbucks"]*20,
               criteria = ["Starbuks"]*20)
gdf_starbucks.head()

In [ ]:
Map(Layer(gdf_starbucks, popup_hover = [popup_element("name","Starbucks in NYC")]))

### 2. Looking for vegan restaurants

In [ ]:
url2 = 'https://api.foursquare.com/v2/venues/explore'
vegan = "4bf58dd8d48988d1d3941735"

In [ ]:
parametros_vegan = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"vegans",
                "radius":500
}

In [ ]:
resp2 = requests.get(url = url_query, params = parametros_vegan)
data2 = json.loads(resp2.text)

In [ ]:
data2.keys()

In [ ]:
decoding_data2 = data2.get("response")

In [ ]:
decoded2 = decoding_data2.get("groups")[0]

In [ ]:
vegan = decoded2.get("items")

In [ ]:
map_vegan = ["venue","name"]
m_latitudvegan = ["venue","location","lat"]
m_longitudvegan = ["venue","location","lng"]

In [ ]:
def getFromDict(diccionario2,mapa2):
    return reduce (operator.getitem,mapa2,diccionario2)

In [ ]:
print(getFromDict(vegan[0],map_vegan))

In [ ]:
vegan_nyc = []
for dic in vegan:
    paralista2 = {}
    paralista2["name"] = getFromDict(dic,map_vegan)
    paralista2["latitud"] = getFromDict(dic,m_latitudvegan)
    paralista2["longitud"] = getFromDict(dic,m_longitudvegan)
    vegan_nyc.append(paralista2)

In [ ]:
vegan_nyc[0]

In [ ]:
df_vegans = pd.DataFrame(vegan_nyc)
df_vegans.head()

In [ ]:
df_vegans[:5]

In [ ]:
gdf_vegan = gpd.GeoDataFrame(df_vegans, geometry = gpd.points_from_xy(df_vegans.longitud,df_vegans.latitud))
gdf_vegan.head()

In [ ]:
gdf_vegan.name

In [ ]:
gdf_vegan = gdf_vegan.assign(name = ["Beyond Sushi",
                                        "by CHLOE.",
                                        "Van Leeuwen Ice Cream",
                                        "Taco Dumbo",
                                        "L’Avenue",
                                        "Taco Dumbo",
                                        "Urbanspace W52",
                                        "The Halal Guys",
                                        "The Little Beet",
                                        "Lenwich by Lenny's",
                                        "Le Pain Quotidien",
                                        "Mysttik Masala",
                                        "Museum of Modern Art (MoMA)",
                                        "Fogo de Chão",
                                        "Aldo Sohm Wine Bar",
                                        "Gregorys Coffee",
                                        "Pret A Manger",
                                        "Devon & Blakely",
                                        "Fig & Olive",
                                        "The Modern",
                                        "Black Tap",
                                        "Simon Sips",
                                        "Barilla Restaurants",
                                        "Estiatorio Milos",
                                        "Le Pain Quotidien",
                                        "Cock & Bull British Pub and Eatery",
                                        "Europa Cafe",
                                        "Forty2West",
                                        "Butter Midtown",
                                        "Natureworks"],
                                criteria = ["vegan restaurant"]*30)
gdf_vegan.head()

In [ ]:
Map(Layer(gdf2, popup_hover = [popup_element("name","Vegan restaurants in NYC")]))

### 3. Looking for a basketball stadium


In [ ]:
url3 = 'https://api.foursquare.com/v2/venues/explore'
basketball_stadium = "4bf58dd8d48988d18b941735"
# Ubicado al mens a 10 km

In [ ]:
parametros_basket = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"basket_stadium",
                "radius":1000
}

In [ ]:
resp3 = requests.get(url = url_query, params = parametros_basket)
data3 = json.loads(resp3.text)

In [ ]:
data3.keys()

In [ ]:
decoding_data3 = data3.get("response")

In [ ]:
decoded3 = decoding_data3.get("groups")[0]

In [ ]:
basket_stadium = decoded3.get("items")

In [ ]:
map_stadium = ["venue","name"]

In [ ]:
m_latitud_stadium = ["venue","location","lat"]
m_longitud_stadium = ["venue","location","lng"]
def getFromDict(diccionario3,mapa3):
    return reduce (operator.getitem,mapa3,diccionario3)

In [ ]:
print(getFromDict(basket_stadium[0],map_stadium))

In [ ]:
basket_stadium_nyc = []
for dic in basket_stadium:
    paralista3 = {}
    paralista3["name"] = getFromDict(dic,map_stadium)
    paralista3["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista3["longitud"] = getFromDict(dic,m_longitud_stadium)
    basket_stadium_nyc.append(paralista3)

In [ ]:
basket_stadium_nyc[0]
df_stadium = pd.DataFrame(basket_stadium_nyc)
df_stadium.head()

In [ ]:
basketball_stadiums_location = list(df_stadium)
basketball_stadiums_location

In [ ]:
gdf_stadiums = gpd.GeoDataFrame(df_stadium, geometry = gpd.points_from_xy(df_stadium.longitud,df_stadium.latitud))
gdf_stadiums

In [ ]:
gdf_stadiums = gdf_stadiums.assign(name = [ "Regal E-Walk 4DX & RPX",
                           "Zaro's Bakery",
                           "Boomer Esiason's Stadium Grill",
                           "Stadium Grill At Bowlmor Lanes"],
               criteria = ["basketball stadium","basketball stadium","basketball stadium","basketball stadium"])
gdf_stadiums.head()

In [ ]:
Map(Layer(gdf_stadiums, popup_hover = [popup_element("name","Basketball Stadium")]))

### 4. Night clubs

In [ ]:
url4 = 'https://api.foursquare.com/v2/venues/explore'
night_clubs = "4bf58dd8d48988d11f941735"

In [ ]:
parametros_night_clubs = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"night_clubs","radius":500
}

resp4 = requests.get(url = url_query, params = parametros_night_clubs)
data4 = json.loads(resp4.text)

In [ ]:
data4.keys()

In [ ]:
decoding_data4 = data4.get("response")

In [ ]:
'''for k,v in decoding_data4.items():
    print(k,v,"\n")'''

In [ ]:
decoded4 = decoding_data4.get("groups")[0]

In [ ]:
night_clubs = decoded4.get("items")

In [ ]:
map_night_clubs = ["venue","name"]

In [ ]:
m_latitud_nightclubs = ["venue","location","lat"]
m_longitud_nightclubbs = ["venue","location","lng"]

In [ ]:
def getFromDict(diccionario4,mapa4):
    return reduce (operator.getitem,mapa4,diccionario4)

In [ ]:
print(getFromDict(night_clubs[0],map_night_clubs))

In [ ]:
night_clubs_nyc = []
for dic in night_clubs:
    paralista4 = {}
    paralista4["name"] = getFromDict(dic,map_night_clubs)
    paralista4["latitud"] = getFromDict(dic,m_latitud_nightclubs)
    paralista4["longitud"] = getFromDict(dic,m_longitud_nightclubbs)
    night_clubs_nyc.append(paralista4)

In [ ]:
night_clubs_nyc[0]
df_night_clubs = pd.DataFrame(night_clubs_nyc)
df_night_clubs.head()

In [ ]:
night_clubs_location = list(df_night_clubs)
night_clubs_location

In [ ]:
gdf_night_clubs = gpd.GeoDataFrame(df_night_clubs, geometry = gpd.points_from_xy(df_night_clubs.longitud,df_night_clubs.latitud))
gdf_night_clubs.head()

In [ ]:
gdf_night_clubs

In [ ]:
gdf_night_clubs = gdf_night_clubs.assign(name = [ "The Tonight Show starring Jimmy Fallon",
                           "Starry Night by Vincent van Gogh",
                           "New York Sports Clubs",
                           "Night Hotel Times Square",
                           "Saturday Night Live Standby Line"],
               criteria = ["night_clubs","night_clubs","night_clubs","night_clubs","night_clubs"])
gdf_night_clubs.head()

In [ ]:
Map(Layer(gdf_night_clubs, popup_hover = [popup_element("name","Night clubs")]))

### 5. Transport availability

In [ ]:
url5 = 'https://api.foursquare.com/v2/venues/explore'
travel_transport = "4d4b7105d754a06379d81259"

In [ ]:
parametros_travel = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{nyc.get('coordinates')[0]},{nyc.get('coordinates')[1]}",
              "query":f"travel_transport",
                ## "radius":1000
                    }

In [ ]:
resp5 = requests.get(url = url_query, params = parametros_travel)
data5 = json.loads(resp5.text)

In [ ]:
data5.keys()

In [ ]:
decoding_data5 = data5.get("response")

In [ ]:
decoded5 = decoding_data5.get("groups")[0]

In [ ]:
transport = decoded5.get("items")
map_transport = ["venue","name"]

In [ ]:
m_latitud_transport = ["venue","location","lat"]
m_longitud_transport = ["venue","location","lng"]
def getFromDict(diccionario5,mapa5):
    return reduce (operator.getitem,mapa5,diccionario5)

In [ ]:
print(getFromDict(transport[0],map_transport))

In [ ]:
transport_nyc = []
for dic in transport:
    paralista5 = {}
    paralista5["name"] = getFromDict(dic,map_stadium)
    paralista5["latitud"] = getFromDict(dic,m_latitud_stadium)
    paralista5["longitud"] = getFromDict(dic,m_longitud_stadium)
    transport_nyc.append(paralista5)

In [ ]:
transport_nyc[0]

In [ ]:
df_transports = pd.DataFrame(transport_nyc)
df_transports.head()

In [ ]:
gdf_transports = gpd.GeoDataFrame(df_transports, geometry = gpd.points_from_xy(df_transports.longitud,df_transports.latitud))
gdf_transports.head()

In [ ]:
gdf_transports = gdf_transports.assign(name = [ "Theater Row - The Acorn",
                           "Solstice Travel Vacations",
                           "The Travel Inn",
                           "United Bus and Travel",
                           "Double Happiness Travel",
                           "K International Transport Co., Inc",
                           "Delgado Travel",
                           "Active Transport ServicesInc.",
                           "Rivas Travel & Multi Services"],
               criteria = ["transports","transports","transports","transports","transports","transports","transports","transports","transports"])
gdf_transports.head()

In [ ]:
Map(Layer(gdf_transports, popup_hover = [popup_element("name","Transports")]))

## Finally, we concatenate our criterias' dataframes.

In [ ]:
df_criterias = pd.concat([df_office,gdf_starbucks, gdf_vegan,gdf_stadiums,gdf_night_clubs,gdf_transports])
df_criterias.head()

In [ ]:
df_criterias = df_criterias